In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glob import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import h5py
import os
from PIL import Image

In [3]:
data_dir = '/Users/nayyarz/WLocalData/rockAI/data/original/'
out_dir = '/Users/nayyarz/WLocalData/rockAI/'
image_size = (224, 224)

In [4]:
filepaths = [f for f in glob(os.path.join(data_dir, '**'), recursive=True)
             if os.path.isfile(f) and os.path.splitext(f)[-1].lower() in ['.jpg', '.png']]
print(filepaths[:5])

['/Users/nayyarz/WPods/pycharm_works/rockAI/data/original/RA/P1033527.JPG', '/Users/nayyarz/WPods/pycharm_works/rockAI/data/original/RA/IMG_5811.JPG', '/Users/nayyarz/WPods/pycharm_works/rockAI/data/original/RA/P1033533.JPG', '/Users/nayyarz/WPods/pycharm_works/rockAI/data/original/RA/P1033269.JPG', '/Users/nayyarz/WPods/pycharm_works/rockAI/data/original/RA/IMG_5150.JPG']


In [5]:
len(filepaths)

1458

In [6]:
_old_s = ''

def print_progress(s):
    
    global _old_s
    print(' ' * len(_old_s), end='\r')
    print(s, end='\r', flush=True)
    _old_s = s

def load_images(paths):
    
    N = len(paths)
    X = np.zeros((N, image_size[0], image_size[1], 3))

    for i, path in enumerate(paths):        
        print_progress(f'[{i+1}/{N}] {path}')
        img = Image.open(path)
        img = img.resize(image_size)
        X[i] = np.array(img)
    
    return X

X = load_images(filepaths)

In [7]:
relpaths = [os.path.relpath(f, data_dir) for f in filepaths]
classnames = [f.split('/', 1)[0] for f in relpaths]
filenames = [f.split('/', 1)[1] for f in relpaths]

In [8]:
np.unique(classnames)

array(['No_RA', 'RA'], dtype='<U5')

In [15]:
len(classnames)

1458

In [16]:
np_classnames = np.array(classnames)
indicies = np.where(np_classnames == '100CANON')[0]
np_classnames[indicies] = 'No_RA'

In [17]:
classnames = np_classnames.tolist()

In [18]:
np.unique(classnames)

array(['No_RA', 'RA'], dtype='<U5')

In [19]:
len(np.where(np_classnames == 'RA')[0])

1339

In [20]:
len(np.where(np_classnames == 'No_RA')[0])

119

In [21]:
h5f = h5py.File(os.path.join(out_dir, f'RockAI_images_{image_size[0]}x{image_size[1]}.h5'), 'w')
h5f['X'] = X
h5f['filename'] = np.array(filenames).astype('S10')
h5f['classname'] = np.array(classnames).astype('S10')
h5f.close()